In [1]:
import json
from datasketch import MinHash, MinHashLSH
import json
from datasketch import MinHash, MinHashLSH
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# 文件路径
input_file_path = '/root/autodl-tmp/dataset/GPT3.5-3270k/3270k-qa.jsonl'
output_file_path = '/root/autodl-tmp/dataset/GPT3.5-3270k/3270k-qa-unique.jsonl'

In [2]:
%%time
# 定义一个函数来处理每一行数据
def process_line(line):
    data = json.loads(line)
    try:
        text = data['conversation'][0]['User'] + data['conversation'][1]['Assistant']
    except:
        text = 'None'
    m = MinHash(num_perm=128)
    for d in text:
        m.update(d.encode('utf8'))
    return m, data

# 读取文件中的所有行
with open(input_file_path, 'r') as file:
    lines = file.readlines()

# 设置进程池
pool = Pool(processes=cpu_count())  # 使用所有可用的CPU核心

# 使用进程池映射处理函数到每一行数据
minhash_data_pairs = pool.map(process_line, lines)

# 关闭进程池并等待所有进程完成
pool.close()
pool.join()

# 在主进程中构建MinHashLSH并插入MinHash对象
lsh = MinHashLSH(threshold=0.6, num_perm=128)
unique_data = []
for m, data in tqdm(minhash_data_pairs):
    # 检查是否有相似的MinHash已经存在
    if lsh.query(m):
        continue  # 如果存在，跳过这个数据
    # 如果不存在，插入MinHash并保存数据
    lsh.insert(str(data), m)
    unique_data.append(data)

# 输出去重结果数量
print(f"Number of unique items: {len(unique_data)}")

# 如果需要，可以将去重结果写入新的JSONL文件
with open(output_file_path, 'w') as outfile:
    for item in unique_data:
        outfile.write(json.dumps(item, ensure_ascii=False) + '\n')

100%|██████████| 3208736/3208736 [02:35<00:00, 20675.67it/s]


Number of unique items: 289803
CPU times: user 5min 1s, sys: 2min 32s, total: 7min 34s
Wall time: 14min 17s


In [10]:
for item in unique_data:
    if item['conversation'][1]['Assistant']=='None':
        print(item)
        break

In [6]:
print(f"Number of unique items: {len(unique_data)}")

Number of unique items: 289803


In [3]:
count=0
with open('/root/autodl-tmp/dataset/GPT3.5-3270k/3270k-qa-.jsonl','r') as file:
    for line in file:
        count+=1
count

289803

In [4]:
count=0
with open('/root/autodl-tmp/dataset/GPT3.5-3270k/3270k-qa.jsonl','r') as file:
    for line in file:
        count+=1
count

3208736